In [69]:
from datetime import datetime
from pprint import pprint
import re
import sqlite3

In [2]:
db = sqlite3.connect('database.sqlite3')

In [24]:
!sqlite3 database.sqlite3 .schema

-- Loading resources from /home/remram/.sqliterc
CREATE TABLE windows(
                id INTEGER PRIMARY KEY,
                start DATETIME NOT NULL,
                end DATETIME NOT NULL,
                active BOOLEAN NOT NULL,
                name TEXT NOT NULL
            );
CREATE TABLE runs(
                id INTEGER PRIMARY KEY,
                start DATETIME NOT NULL,
                end DATETIME NULL,
                end_reason TEXT NOT NULL DEFAULT ''
            );
CREATE INDEX idx_windows_start ON windows(start);
CREATE INDEX idx_windows_end ON windows(end);
CREATE INDEX idx_windows_active ON windows(active) WHERE active=1;
CREATE INDEX idx_windows_name ON windows(name);
CREATE INDEX idx_runs_start ON runs(start);
CREATE INDEX idx_runs_end ON runs(end);


In [3]:
list(db.execute('SELECT count(id) FROM windows;'))

[(2100,)]

In [9]:
names = [name for name, in db.execute('SELECT DISTINCT name FROM windows;')]
len(names)

141

In [10]:
for name in sorted(names):
    print(name)

"Command Line Tools" missing find · Issue #22 · remram44/regex-cheatsheet — Mozilla Firefox
"database is locked" issue (#47) · Issues · VIDA-NYU / d3m / AlphaD3M · GitLab — Mozilla Firefox
() datamart-prod — Konsole
(NSFW) What Fetish did you have until you tried it? : AskReddit — Mozilla Firefox
- : sudo -g — Konsole
/home/remram/projects/d3m/auctus - QGit
/home/remram/projects/pctracker - QGit
/home/remram/projects/peertube - QGit
2017 IRONMAN Lake Placid » RTRT.me — Mozilla Firefox
2020-12-17 15-00-28.mkv - VLC media player
ASMR Cranial Nerve Exam: Concussion Test 👩‍⚕️ (Soft Spoken Roleplay - Doctor Check Up ASMR) - YouTube — Mozilla Firefox
Add Comment | Hacker News — Mozilla Firefox
Add facets to filters (!179) · Merge Requests · VIDA-NYU / auctus / Auctus · GitLab — Mozilla Firefox
Aposimz chapter 3 vol — Mozilla Firefox
Aposimz chapter 30 vol — Mozilla Firefox
Auctus — Mozilla Firefox
Autostart — System Settings Module
Back to the '70s with Serverless — Mozilla Firefox
Beer dist

In [108]:
def regex(pattern):
    compiled = re.compile(pattern)
    
    def regex_matcher(record):
        return compiled.search(record.name) is not None
    
    return repr(pattern), regex_matcher

In [109]:
filters = [
    (regex(' — Mozilla Firefox$'), [
        (regex(' - YouTube — Mozilla Firefox$'), []),
    ]),
    (regex(' - GVIM$'), []),
]

In [110]:
class Record(object):
    def __init__(self, start, end, name):
        self.start = datetime.fromisoformat(start)
        self.end = datetime.fromisoformat(end)
        self.name = name
        
    @property
    def duration(self):
        return (self.end - self.start).total_seconds()

In [142]:
class OutputNode(object):
    def __init__(self, name):
        self.name = name
        self.duration = 0
        self.children = {}
        
    def child(self, name):
        try:
            child = self.children[name]
        except KeyError:
            child = self.children[name] = OutputNode(name)
        return child
    
    def print(self, indent=0):
        if indent == 0:
            prefix = ''
            last_prefix = '└── '
        else:
            prefix = '│   ' * (indent - 1)
            prefix += '├── '
            last_prefix = '|   ' * indent
            last_prefix += '└── '
        name = self.name or 'total'
        other_duration = self.duration
        duration = '%ds' % int(self.duration)
        print(f'{prefix}{name} {duration}')
        for child in self.children.values():
            child.print(indent + 1)
            other_duration -= child.duration
        print(f'{last_prefix}other {other_duration}')

In [143]:
def apply_filters(record, filters, output):
    # Add duration to current node
    output.duration += record.duration
    # Apply filters until one matches
    for (name, matcher), then_ops in filters:
        if matcher(record):
            apply_filters(record, then_ops, output.child(name))
            return


output = OutputNode(None)

for row in db.execute('''\
    SELECT start, end, name
    FROM windows
    WHERE active=1;
'''):
    record = Record(*row)
    apply_filters(record, filters, output)

In [144]:
output.print()

total 7737s
├── ' — Mozilla Firefox$' 5988s
│   ├── ' - YouTube — Mozilla Firefox$' 1166s
|   |   └── other 1166.0
|   └── other 4822.0
├── ' - GVIM$' 86s
|   └── other 86.0
└── other 1663.0
